In [ ]:
import pandas as pd
import json

In [ ]:
# read in bussiness json, subset to only PA
rest_data = []
with open("data/yelp_dataset/yelp_academic_dataset_business.json", 'r') as data_file:
    for line in data_file:
        rest_data.append(json.loads(line))

rest_df = pd.DataFrame(rest_data)
rest_df
pa_rests = rest_df[rest_df['state'] == 'PA']

In [ ]:
# inspections data
phila_inspecs = pd.read_csv('data/phila/phila_inspections.csv')
phila_inspecs

In [ ]:
# drop rows with missing labels
clean_phila = phila_inspecs.dropna(subset=['Overall Compliance'])

In [ ]:
# pre-merge cleaning
clean_phila['Address'] = clean_phila['Address'].astype(str)
clean_phila['Address'] = clean_phila['Address'].str.strip()
clean_phila['Address'] = clean_phila['Address'].str.upper()

pa_rests['address'] = pa_rests['address'].astype(str)
pa_rests['address'] = pa_rests['address'].str.strip()
pa_rests['address'] = pa_rests['address'].str.upper()

In [ ]:
labeled = pd.merge(clean_phila, pa_rests, how='left', left_on='Address', right_on='address').dropna()
labeled['Overall Compliance'].value_counts()

In [ ]:
# get lists of labled resturant ids
labeled_ids = list(labeled['business_id'])

In [ ]:
# read in review text data
review_data = []
with open("data/yelp_dataset/yelp_academic_dataset_review.json", 'r') as data_file:
    for line in data_file:
        review_data.append(json.loads(line))
review_df = pd.DataFrame(review_data)

In [ ]:
#collect reviews in inspection period
labeled_reviews = review_df[review_df['business_id'].isin(labeled_ids)]
labeled_reviews['date'] = pd.to_datetime(labeled_reviews['date'])
labeled_reviews = labeled_reviews[(labeled_reviews['date']  >= '2018-01-01') & (labeled_reviews['date'] < '2020-12-31')]